In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import style
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import datetime
from datetime import timedelta
import random
import hashlib
import collections

style.use('ggplot')
%matplotlib inline

naver_dictionary = {}    
def naver_crawling_related_word(word_list, n):
    
    if n==0:
        print("크롤링 종료")
    else:
        merge_df = pd.DataFrame()

        for i in range(len(word_list)):
            temp_df = pd.DataFrame()
            temp_list = []

            url = f'https://search.naver.com/search.naver?where=nexearch&query={word_list[i]}'

            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')

            ul_inner = soup.find('ul', {'class':'lst_related_srch _list_box'})

            try:
                keyword_list = soup.find('ul', {'class':'lst_related_srch _list_box'}).find_all('li')
            except:
                if pd.isna(ul_inner):
                    try:
                        keyword_list = soup.find('ul', {'class':'lst_related_srch _list_box'}).find_all('li')
                    except:
                        keyword_list = []

            # span_list 가 공백일 때 예외 처리
            if len(keyword_list) == 0:
                pass
            else:
                for k, word in enumerate(keyword_list):
                    if len(keyword_list[k].text.replace(word_list[i],'').strip()) == 1:
                        temp_list.append(keyword_list[k].text.strip())
                    else:
                        temp_list.append(keyword_list[k].text.strip())
                        
                temp_df['sub_word'] = temp_list
                temp_df['sup_word'] = word_list[i]
                temp_df = pd.DataFrame(temp_df, columns=['sup_word', 'sub_word'])

                merge_df = pd.concat([merge_df, temp_df], axis=0)
        naver_dictionary[n] = merge_df

        # 재귀호출
        pass_word_list = merge_df['sub_word'].values

        print('네이버 크롤링 시작')
        naver_crawling_related_word(pass_word_list, n-1)

        
daum_dictionary = {}

def daum_crawling_related_word(word_list, n):
    
    if n==0:
        print("크롤링 종료")
    else:
        merge_df = pd.DataFrame()

        for i in range(len(word_list)):
            temp_df = pd.DataFrame()
            temp_list = []

            url = f'https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&q={word_list[i]}'

            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')

            div_inner = soup.find('div', {'class':'list_keyword type2'})

            try:
                div_inner = soup.find('div', {'class':'list_keyword type2'})
                span_list = div_inner.find_all('span', {'class':'wsn'})
            except:
                if pd.isna(div_inner):
                    try:
                        div_inner = soup.find('div', {'class':'list_keyword'})
                        span_list = div_inner.find_all('span', {'class':'wsn'})
                    except:
                        span_list = []

            # span_list 가 공백일 때 예외 처리
            if len(span_list) == 0:
                pass
            else:
                for k, word in enumerate(span_list):
                    if len(span_list[k].text.replace(word_list[i],'').strip()) == 1:
                        temp_list.append(span_list[k].text.strip())
                    else:
                        temp_list.append(span_list[k].text.strip())

                temp_df['sub_word'] = temp_list
                temp_df['sup_word'] = word_list[i]
                temp_df = pd.DataFrame(temp_df, columns=['sup_word', 'sub_word'])

                merge_df = pd.concat([merge_df, temp_df], axis=0)

        daum_dictionary[n] = merge_df

        # 재귀호출
        pass_word_list = merge_df['sub_word'].values
        print('다음 크롤링 시작')
        daum_crawling_related_word(pass_word_list, n-1)    

word_list = ['메타버스']

naver_crawling_related_word(word_list, 2)
daum_crawling_related_word(word_list, 2)

daum_dictionary[2].columns = ['sup_word', 'sub_word1']
daum_dictionary[1].columns = ['sub_word1', 'sub_word2']

naver_dictionary[2].columns = ['sup_word', 'sub_word1']
naver_dictionary[1].columns = ['sub_word1', 'sub_word2']

daum_td = pd.merge(left = daum_dictionary[2],right = daum_dictionary[1], how = "left", on = "sub_word1")
naver_td = pd.merge(left = naver_dictionary[2],right = naver_dictionary[1], how = "left", on = "sub_word1")
total_df = pd.concat([daum_td, naver_td])
total_df = total_df.reset_index(drop=True)
total_df

네이버 크롤링 시작
네이버 크롤링 시작
크롤링 종료
다음 크롤링 시작
다음 크롤링 시작
크롤링 종료


,sup_word,sub_word1,sub_word2
0,메타버스,메타버스란,메타버스 관련주
1,메타버스,메타버스란,메타버스 뜻
2,메타버스,메타버스란,메타버스 대장주
3,메타버스,메타버스란,메타버스 플랫폼
4,메타버스,메타버스란,메타버스 etf
...,...,...,...
429,메타버스,네이버 메타버스,메타버스 게임
430,메타버스,네이버 메타버스,메타버스 제페토
431,메타버스,네이버 메타버스,네이버z
432,메타버스,네이버 메타버스,네이버랩스


In [2]:
word_list = ['엑시인피니티']

naver_crawling_related_word(word_list, 2)
daum_crawling_related_word(word_list, 2)

daum_dictionary[2].columns = ['sup_word', 'sub_word1']
daum_dictionary[1].columns = ['sub_word1', 'sub_word2']

naver_dictionary[2].columns = ['sup_word', 'sub_word1']
naver_dictionary[1].columns = ['sub_word1', 'sub_word2']

daum_td = pd.merge(left = daum_dictionary[2],right = daum_dictionary[1], how = "left", on = "sub_word1")
naver_td = pd.merge(left = naver_dictionary[2],right = naver_dictionary[1], how = "left", on = "sub_word1")
total_df = pd.concat([daum_td, naver_td])
total_df = total_df.reset_index(drop=True)
total_df

네이버 크롤링 시작
네이버 크롤링 시작
크롤링 종료
다음 크롤링 시작
다음 크롤링 시작
크롤링 종료


,sup_word,sub_word1,sub_word2
0,엑시인피니티,엑시인피니티 전망,엑시인피니티 시세
1,엑시인피니티,엑시인피니티 전망,엑시 코인
2,엑시인피니티,엑시인피니티 전망,인피니티 2021
3,엑시인피니티,엑시인피니티 전망,엑시인피니티 호재
4,엑시인피니티,엑시인피니티 전망,엑시인피니티 목표가
...,...,...,...
273,엑시인피니티,삼성 엑시인피니티,NaN
274,엑시인피니티,엑시인피니티 락업해제,NaN
275,엑시인피니티,엑시인피니티 모바일,엑시인피니티
276,엑시인피니티,엑시인피니티 필리핀,엑시인피니티


In [3]:
import hashlib
import hmac
import base64

import requests
import time
import random
import requests
from powernad.API import RelKwdStat
import pandas as pd
BASE_URL = 'https://api.naver.com'
API_KEY = '0100000000f93df373446ea1ee430f49c14690f1f0ec1ae6a624fa652419ab52ef231273f0'
SECRET_KEY = 'AQAAAAD5PfNzRG6h7kMPScFGkPHwvuPYo+dmLIk2xT+T3eUgyg=='
CUSTOMER_ID = '2414867'

def naverKwdApi(keyword):
    data_list = []
    
    relKwdStat = RelKwdStat.RelKwdStat(BASE_URL, API_KEY, SECRET_KEY, CUSTOMER_ID)
    
    kwdDataList = relKwdStat.get_rel_kwd_stat_list(None, hintKeywords=keyword, showDetail='1')
    
    for outdata in kwdDataList:
        relKeyword = outdata.relKeyword # 연관 키워드
        monthleyPcQcCnt = outdata.monthlyPcQcCnt # 30일간 PC 조회수
        monthleyMobileQcCnt = outdata.monthlyMobileQcCnt # 30일간 모바일 조회수
        monthlyAveMobileClkCnt = outdata.monthlyAveMobileClkCnt # 4주간 평균 모바일 클릭수
        monthlyAvePcCtr = outdata.monthlyAvePcCtr # 4주간 평균 PC 클릭율
        monthlyAveMobileCtr = outdata.monthlyAveMobileCtr # 4주간 평균 모바일 클릭율
        plAvgDepth = outdata.plAvgDepth # 4주간 평균 PC 광고수
        compIdx = outdata.compIdx
        
        if monthleyPcQcCnt == '< 10':
            monthleyPcQcCnt = 0
        
        if monthleyMobileQcCnt == '< 10':
            monthleyMobileQcCnt = 0
            
#         print(relKeyword,monthleyMobileQcCnt)
        json_obj = {
            '키워드' : keyword,
            '연관 키워드':relKeyword,
            '월간 PC 검색수':int(monthleyPcQcCnt),
            '월간 Mobile 검색수':int(monthleyMobileQcCnt),
            '검색수합계': int(monthleyPcQcCnt)+int(monthleyMobileQcCnt),
            '월평균 PC 클릭수' : int(monthlyAvePcCtr),
            '월평균 Mobile 클릭수' : int(monthlyAveMobileClkCnt),
            '월평균 PC 클릭율': monthlyAvePcCtr,
            '월평균 Mobile 클릭율':monthlyAveMobileCtr,
            '경쟁정도':compIdx,
            '월평균노출광고수':plAvgDepth
        }
        data_list.append(json_obj)

    df = pd.DataFrame(data_list)
    return df


In [ ]:
naverKwdApi("메타버스")

In [ ]:
total_df

In [ ]:
df = naverKwdApi("메타버스")

df

In [ ]:
naverKwdApi("전자책강의")

In [ ]:
df = naverKwdApi("더존비즈온")

df=df[['키워드','연관 키워드', '검색수합계']]
df.columns = ['sup_word', 'sub_word', 'views']
df

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib 

import networkx as nx
import matplotlib.font_manager as fm


def netword_drawing(total_words):
    print("그리기 시작")
    G = nx.Graph()
    
    for sup in list(set(total_words['sup_word'].values)):
        G.add_node(sup)
        temp_df1 = total_words[total_words['sup_word']==sup]
        
        for sub1 in list(set(temp_df1['sub_word'].values)):
            G.add_node(sub1)
            G.add_edge(sup, sub1)
#             temp_df2 = temp_df1[temp_df1['sub_word']==sub1]
            
#             for sub2 in list(set(temp_df2['sub_word2'].values)):
#                 G.add_node(sub2)
#                 G.add_edge(sub1, sub2)
                
    color_map = []
    for node in G:
        if node == total_words['sup_word'].values[0]:
            color_map.append('red')
        else:
            color_map.append('green')
    
    plt.figure(figsize=(100,100))
#     ㅇ = dict(G.degree)
    pos=nx.shell_layout(G)
    d = dict(zip(total_words['sub_word'],total_words['views']))
#     print(d)
    print('-')
    nx.draw(G, nodelist=list(d), node_size=[v for v in d.values()], font_family='AppleGothic', font_size=20,node_color = color_map, with_labels = True)

    plt.axis('off')
    plt.savefig('networx_picture_'+total_words['sup_word'].values[0]+'.png')
    plt.show()

netword_drawing(df.loc[:len(df)])

In [ ]:
df.loc[1:]

In [ ]:
naverKwdApi("코로케이션")

In [ ]:
total_df

In [3]:
from neo4j import GraphDatabase

total_df = total_df.fillna('없음')

total_df.to_csv('/Users/kimminho/Downloads/neo4j-community-3.5.30/import/neo_test.csv')

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "1234"), encrypted=False)

def add_keyword_node(tx):
    re_sub_query = '''
        LOAD CSV WITH HEADERS FROM "file:///neo_test.csv" AS row
        MERGE (s:Sup_word {sup_word: row.sup_word})
        WITH s, row
        MERGE(m:Sub_word1 {sub_word1:row.sub_word1})
        MERGE (s) - [r:Depth1] -> (m)
        MERGE(e:Sub_word2 {sub_word2:row.sub_word2})
        MERGE (m) - [r2:Depth2] -> (e)  
        
    '''
    tx.run(re_sub_query)

def removeExistingData(connection):
    '''
    Neo4j does not replace a dataset when additional data is added, that is
    why we need to remove all existing data before loading the dataset again
    '''
    connection.run("MATCH (n) DETACH DELETE n")


with driver.session() as session:
      session.write_transaction(removeExistingData)
      session.write_transaction(add_keyword_node)


driver.close()



Transaction failed and will be retried in 1.0742751872434044s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 61] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 61] Connection refused))
Transaction failed and will be retried in 1.8559891389177299s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 61] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 61] Connection refused))
Transaction failed and will be retried in 3.556438829042472s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 61] Connection 

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 61] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 61] Connection refused)

# Interactiv Network Graph

In [ ]:
!pip3 install bokeh

In [ ]:
import re
s = "asd &수고했어요! "
body = re.sub(r"[^0-9가-힣?.!,¿]+", " ", s)
body

In [ ]:
import numpy as np
from bokeh.io import output_notebook, show, save

output_notebook()

In [ ]:
df

In [ ]:
G = nx.from_pandas_edgelist(df, 'sup_word', 'sub_word', 'views')

In [ ]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

In [ ]:
degrees = dict(zip(df['sub_word'],df['views']))
nx.set_node_attributes(G, name='degree', values=degrees)

In [ ]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [ ]:
#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Relatet Keyword Network'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Character", "@index"),
        ("Degree", "@degree")
]
#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html\
network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")